In [1]:
# for presentation purposes
import warnings
warnings.filterwarnings("ignore")

# wrangle
import os

# transform
import numpy as np
import pandas as pd

# visualize 
import matplotlib.pyplot as plt
import seaborn as sns

# working with dates
from datetime import datetime

# modeling
import statsmodels.api as sm
from statsmodels.tsa.api import Holt, ExponentialSmoothing

# evaluate
from sklearn.metrics import mean_squared_error
from math import sqrt 

In [2]:
def get_mental_health_data():
    if os.path.isfile('mental_health_data.csv'):
        df = pd.read_csv('mental_health_data.csv')
        return df
    else:
        print('Please save the .csv file locally from Kaggle.')

In [3]:
df = get_mental_health_data()

In [4]:
df.isna().sum()

index                             0
Entity                            0
Code                           5412
Year                              0
Schizophrenia (%)             82678
Bipolar disorder (%)          89147
Eating disorders (%)           8317
Anxiety disorders (%)        102085
Drug use disorders (%)       102085
Depression (%)               102085
Alcohol use disorders (%)    102085
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108553 entries, 0 to 108552
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   index                      108553 non-null  int64  
 1   Entity                     108553 non-null  object 
 2   Code                       103141 non-null  object 
 3   Year                       108553 non-null  object 
 4   Schizophrenia (%)          25875 non-null   object 
 5   Bipolar disorder (%)       19406 non-null   object 
 6   Eating disorders (%)       100236 non-null  object 
 7   Anxiety disorders (%)      6468 non-null    float64
 8   Drug use disorders (%)     6468 non-null    float64
 9   Depression (%)             6468 non-null    float64
 10  Alcohol use disorders (%)  6468 non-null    float64
dtypes: float64(4), int64(1), object(6)
memory usage: 9.1+ MB


In [6]:
df.head()

,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [7]:
import summarize as s

In [8]:
# summarizing the data and checking for nulls
s.summarize(df)

                    SUMMARY REPORT


Dataframe head: 


,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644




Dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108553 entries, 0 to 108552
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   index                      108553 non-null  int64  
 1   Entity                     108553 non-null  object 
 2   Code                       103141 non-null  object 
 3   Year                       108553 non-null  object 
 4   Schizophrenia (%)          25875 non-null   object 
 5   Bipolar disorder (%)       19406 non-null   object 
 6   Eating disorders (%)       100236 non-null  object 
 7   Anxiety disorders (%)      6468 non-null    float64
 8   Drug use disorders (%)     6468 non-null    float64
 9   Depression (%)             6468 non-null    float64
 10  Alcohol use disorders (%)  6468 non-null    float64
dtypes: float64(4), int64(1), object(6)
memory usage: 9.1+ MB


""




Dataframe Description: 


...

,Alcohol use disorders (%)
"(0.441, 0.95]",1421
"(0.95, 1.452]",1675
"(1.452, 1.955]",1928
"(1.955, 2.458]",857
"(2.458, 2.961]",218
"(2.961, 3.464]",87
"(3.464, 3.966]",78
"(3.966, 4.469]",22
"(4.469, 4.972]",73
"(4.972, 5.475]",109


nulls in dataframe by column: 


,num_rows_missing,percent_rows_missing
Anxiety disorders (%),102085,94.041620
Drug use disorders (%),102085,94.041620
Depression (%),102085,94.041620
Alcohol use disorders (%),102085,94.041620
Bipolar disorder (%),89147,82.123018
Schizophrenia (%),82678,76.163717
Eating disorders (%),8317,7.661695
Code,5412,4.985583
index,0,0.000000
Entity,0,0.000000


nulls in dataframe by row: 


,num_cols_missing,percent_cols_missing
56449,7,63.636364
76743,7,63.636364
76751,7,63.636364
76750,7,63.636364
76749,7,63.636364
...,...,...
4072,0,0.000000
4073,0,0.000000
4074,0,0.000000
4075,0,0.000000


In [9]:
df.columns

Index(['index', 'Entity', 'Code', 'Year', 'Schizophrenia (%)',
       'Bipolar disorder (%)', 'Eating disorders (%)', 'Anxiety disorders (%)',
       'Drug use disorders (%)', 'Depression (%)',
       'Alcohol use disorders (%)'],
      dtype='object')

In [10]:
# renaming columns
df = df.rename(columns={
                'Entity': 'entity',
               'Year': 'year', 
               'Code': 'code', 
               'Schizophrenia (%)': 'schizophrenia',
               'Bipolar disorder (%)': 'bipolar_disorder',
               'Eating disorders (%)': 'eating_disorders',
               'Anxiety disorders (%)': 'anxiety_disorders',
               'Drug use disorders (%)': 'drug_use_disorders',
               'Depression (%)': 'depression',
               'Alcohol use disorders (%)': 'alcohol_use_disorders'})

In [11]:
df.head()

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [12]:
len(df)

108553

In [13]:
df.anxiety_disorders.isna().sum()

102085

In [14]:
df.entity.value_counts()

Afghanistan               496
Kazakhstan                496
Morocco                   496
Albania                   496
Myanmar                   496
                         ... 
South Asia                112
Caribbean                 112
Central Asia              112
Southern Latin America    112
Entity                      3
Name: entity, Length: 276, dtype: int64

In [15]:
# There seems to be four different tables within this table

In [16]:
mental_health_df = df[:6468]
mental_health_df

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260
...,...,...,...,...,...,...,...,...,...,...,...
6463,6463,Zimbabwe,ZWE,2013,0.15567,0.607993,0.117248,3.090168,0.766280,3.128192,1.515641
6464,6464,Zimbabwe,ZWE,2014,0.155993,0.60861,0.118073,3.093964,0.768914,3.140290,1.515470
6465,6465,Zimbabwe,ZWE,2015,0.156465,0.609363,0.11947,3.098687,0.771802,3.155710,1.514751
6466,6466,Zimbabwe,ZWE,2016,0.157111,0.610234,0.121456,3.104294,0.772275,3.174134,1.513269


In [17]:
df[df.entity == 'Entity']

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
6468,6468,Entity,Code,Year,Prevalence in males (%),Prevalence in females (%),Population,NaN,NaN,NaN,NaN
54276,54276,Entity,Code,Year,"Suicide rate (deaths per 100,000 individuals)",Depressive disorder rates (number suffering pe...,Population,NaN,NaN,NaN,NaN
102084,102084,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both ...,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
population_df = df[6468:54276]
population_df

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
6468,6468,Entity,Code,Year,Prevalence in males (%),Prevalence in females (%),Population,NaN,NaN,NaN,NaN
6469,6469,Afghanistan,AFG,1800,NaN,NaN,3280000,NaN,NaN,NaN,NaN
6470,6470,Afghanistan,AFG,1801,NaN,NaN,3280000,NaN,NaN,NaN,NaN
6471,6471,Afghanistan,AFG,1802,NaN,NaN,3280000,NaN,NaN,NaN,NaN
6472,6472,Afghanistan,AFG,1803,NaN,NaN,3280000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
54271,54271,Zimbabwe,ZWE,2015,2.789152,3.455323,13815000.000000,NaN,NaN,NaN,NaN
54272,54272,Zimbabwe,ZWE,2016,2.799308,3.479071,14030000.000000,NaN,NaN,NaN,NaN
54273,54273,Zimbabwe,ZWE,2017,2.812022,3.50086,14237000.000000,NaN,NaN,NaN,NaN
54274,54274,Zimbabwe,ZWE,2018,NaN,NaN,14439000.000000,NaN,NaN,NaN,NaN


In [19]:
rates_df = df[54276:102084]
rates_df

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
54276,54276,Entity,Code,Year,"Suicide rate (deaths per 100,000 individuals)",Depressive disorder rates (number suffering pe...,Population,NaN,NaN,NaN,NaN
54277,54277,Afghanistan,AFG,1800,NaN,NaN,3280000,NaN,NaN,NaN,NaN
54278,54278,Afghanistan,AFG,1801,NaN,NaN,3280000,NaN,NaN,NaN,NaN
54279,54279,Afghanistan,AFG,1802,NaN,NaN,3280000,NaN,NaN,NaN,NaN
54280,54280,Afghanistan,AFG,1803,NaN,NaN,3280000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
102079,102079,Zimbabwe,ZWE,2015,27.197061,3068.250731,13815000.0,NaN,NaN,NaN,NaN
102080,102080,Zimbabwe,ZWE,2016,26.839591,3081.782858,14030000.0,NaN,NaN,NaN,NaN
102081,102081,Zimbabwe,ZWE,2017,26.391769,3094.795065,14237000.0,NaN,NaN,NaN,NaN
102082,102082,Zimbabwe,ZWE,2018,NaN,NaN,14439000.0,NaN,NaN,NaN,NaN


In [20]:
depressive_rates_df = df[102084:]
depressive_rates_df

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
102084,102084,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both ...,NaN,NaN,NaN,NaN,NaN,NaN
102085,102085,Afghanistan,AFG,1990,318435.81367,NaN,NaN,NaN,NaN,NaN,NaN
102086,102086,Afghanistan,AFG,1991,329044.773956,NaN,NaN,NaN,NaN,NaN,NaN
102087,102087,Afghanistan,AFG,1992,382544.572895,NaN,NaN,NaN,NaN,NaN,NaN
102088,102088,Afghanistan,AFG,1993,440381.507393,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
108548,108548,Zimbabwe,ZWE,2013,303564.60359,NaN,NaN,NaN,NaN,NaN,NaN
108549,108549,Zimbabwe,ZWE,2014,311665.769283,NaN,NaN,NaN,NaN,NaN,NaN
108550,108550,Zimbabwe,ZWE,2015,320638.507158,NaN,NaN,NaN,NaN,NaN,NaN
108551,108551,Zimbabwe,ZWE,2016,330437.353798,NaN,NaN,NaN,NaN,NaN,NaN


The four dataframes have been separated.

    1. mental_health_df
    2. population_df
    3. rates_df
    4. depressive_rates
    
Lets clean them up.

In [21]:
mental_health_df = mental_health_df.drop(columns='index')

In [22]:
s.summarize(mental_health_df)

                    SUMMARY REPORT


Dataframe head: 


,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644




Dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6468 entries, 0 to 6467
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   entity                 6468 non-null   object 
 1   code                   5488 non-null   object 
 2   year                   6468 non-null   object 
 3   schizophrenia          6468 non-null   object 
 4   bipolar_disorder       6468 non-null   object 
 5   eating_disorders       6468 non-null   object 
 6   anxiety_disorders      6468 non-null   float64
 7   drug_use_disorders     6468 non-null   float64
 8   depression             6468 non-null   float64
 9   alcohol_use_disorders  6468 non-null   float64
dtypes: float64(4), object(6)
memory usage: 505.4+ KB


""




Dataframe Description: 


...

,alcohol_use_disorders
"(0.441, 0.95]",1421
"(0.95, 1.452]",1675
"(1.452, 1.955]",1928
"(1.955, 2.458]",857
"(2.458, 2.961]",218
"(2.961, 3.464]",87
"(3.464, 3.966]",78
"(3.966, 4.469]",22
"(4.469, 4.972]",73
"(4.972, 5.475]",109


nulls in dataframe by column: 


,num_rows_missing,percent_rows_missing
code,980,15.151515
entity,0,0.000000
year,0,0.000000
schizophrenia,0,0.000000
bipolar_disorder,0,0.000000
eating_disorders,0,0.000000
anxiety_disorders,0,0.000000
drug_use_disorders,0,0.000000
depression,0,0.000000
alcohol_use_disorders,0,0.000000


nulls in dataframe by row: 


,num_cols_missing,percent_cols_missing
1750,1,10.0
1183,1,10.0
1191,1,10.0
1190,1,10.0
1189,1,10.0
...,...,...
2312,0,0.0
2311,0,0.0
2310,0,0.0
2309,0,0.0


In [23]:
# This dataset looks pretty clean now as far as null values are concerned, but lets
# check out the nulls in the code column

In [24]:
mental_health_df[mental_health_df.code.isna()]

,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
112,Andean Latin America,NaN,1990,0.195471,0.870564,0.28782,4.217677,0.608921,2.621781,1.748912
113,Andean Latin America,NaN,1991,0.195484,0.870876,0.289362,4.220261,0.610489,2.631717,1.743910
114,Andean Latin America,NaN,1992,0.195497,0.871174,0.290563,4.222587,0.611990,2.639932,1.738703
115,Andean Latin America,NaN,1993,0.195523,0.871482,0.291833,4.224721,0.613995,2.646096,1.733697
116,Andean Latin America,NaN,1994,0.195569,0.871823,0.293111,4.226690,0.616448,2.651022,1.729266
...,...,...,...,...,...,...,...,...,...,...
6351,Western Sub-Saharan Africa,NaN,2013,0.173905,0.635737,0.136469,2.899641,0.477964,3.805780,0.966526
6352,Western Sub-Saharan Africa,NaN,2014,0.174254,0.636185,0.138116,2.902038,0.480415,3.787750,0.967383
6353,Western Sub-Saharan Africa,NaN,2015,0.174616,0.636682,0.139784,2.904311,0.482921,3.762850,0.968057
6354,Western Sub-Saharan Africa,NaN,2016,0.17498,0.637211,0.141193,2.906350,0.485547,3.733871,0.968472


In [25]:
(mental_health_df[mental_health_df.code.isna()]).entity.value_counts()

Andean Latin America                                28
Southeast Asia                                      28
North Africa and Middle East                        28
North America                                       28
Northern Ireland                                    28
Oceania                                             28
Scotland                                            28
South Asia                                          28
Southeast Asia, East Asia, and Oceania              28
Low-middle SDI                                      28
Southern Latin America                              28
Southern Sub-Saharan Africa                         28
Sub-Saharan Africa                                  28
Tropical Latin America                              28
Wales                                               28
Western Europe                                      28
Middle SDI                                          28
Low SDI                                             28
Australasi

In [26]:
(mental_health_df.entity.value_counts() != 28).sum()

# so all values in entity have 28 entries

0

In [27]:
len(df.entity.unique())

# There are only 196 countries in the world

# For now I will leave in the entities which are not countries with the understanding 
# that I wish to aggregte by year so the averages will not be affected as they represent 
# averages of regions (assumption)

276

In [28]:
population_df

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
6468,6468,Entity,Code,Year,Prevalence in males (%),Prevalence in females (%),Population,NaN,NaN,NaN,NaN
6469,6469,Afghanistan,AFG,1800,NaN,NaN,3280000,NaN,NaN,NaN,NaN
6470,6470,Afghanistan,AFG,1801,NaN,NaN,3280000,NaN,NaN,NaN,NaN
6471,6471,Afghanistan,AFG,1802,NaN,NaN,3280000,NaN,NaN,NaN,NaN
6472,6472,Afghanistan,AFG,1803,NaN,NaN,3280000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
54271,54271,Zimbabwe,ZWE,2015,2.789152,3.455323,13815000.000000,NaN,NaN,NaN,NaN
54272,54272,Zimbabwe,ZWE,2016,2.799308,3.479071,14030000.000000,NaN,NaN,NaN,NaN
54273,54273,Zimbabwe,ZWE,2017,2.812022,3.50086,14237000.000000,NaN,NaN,NaN,NaN
54274,54274,Zimbabwe,ZWE,2018,NaN,NaN,14439000.000000,NaN,NaN,NaN,NaN


In [29]:
for col in population_df.columns:
    print(f'There are {population_df[col].isna().sum()}, {round(((population_df[col].isna().sum()) / len(population_df) * 100), 2)}%, null values in {col}')

There are 0, 0.0%, null values in index
There are 0, 0.0%, null values in entity
There are 1726, 3.61%, null values in code
There are 0, 0.0%, null values in year
There are 41339, 86.47%, null values in schizophrenia
There are 41339, 86.47%, null values in bipolar_disorder
There are 924, 1.93%, null values in eating_disorders
There are 47808, 100.0%, null values in anxiety_disorders
There are 47808, 100.0%, null values in drug_use_disorders
There are 47808, 100.0%, null values in depression
There are 47808, 100.0%, null values in alcohol_use_disorders


In [30]:
population_df = population_df.drop(columns={'anxiety_disorders',
                                            'drug_use_disorders',
                                           'depression',
                                           'alcohol_use_disorders'})

In [31]:
population_df = population_df.rename(columns={'schizophrenia': 'prevalence_males', 
                              'bipolar_disorder': 'prevalance_female',
                             'eating_disorders': 'population'})

In [32]:
population_df = population_df.drop(6468)

In [33]:
population_df.isna().sum()

index                    0
entity                   0
code                  1726
year                     0
prevalence_males     41339
prevalance_female    41339
population             924
dtype: int64

In [34]:
population_df[population_df.year == population_df.year.min()]

,index,entity,code,year,prevalence_males,prevalance_female,population
6699,6699,Africa,NaN,0,NaN,NaN,15185668.000000
8599,8599,Asia,NaN,0,NaN,NaN,121526576.000000
20299,20299,Europe,NaN,0,NaN,NaN,38492728.000000
28863,28863,Latin America,NaN,0,NaN,NaN,11981719.000000
37245,37245,North America,NaN,0,NaN,NaN,724947
37905,37905,Oceania,NaN,0,NaN,NaN,327451


In [35]:
population_df[population_df.year == population_df.year.max()]

,index,entity,code,year,prevalence_males,prevalance_female,population
6690,6690,Africa,NaN,9000 BCE,NaN,NaN,342782
8590,8590,Asia,NaN,9000 BCE,NaN,NaN,1557637
20290,20290,Europe,NaN,9000 BCE,NaN,NaN,997425
28854,28854,Latin America,NaN,9000 BCE,NaN,NaN,362548
37236,37236,North America,NaN,9000 BCE,NaN,NaN,52001
37896,37896,Oceania,NaN,9000 BCE,NaN,NaN,252015
53466,53466,World,OWID_WRL,9000 BCE,NaN,NaN,3564407


In [36]:
# In an attempt to get rid of the null values, I may trim down the dataset to only
# include dates with a range from the mental_health_df to also try and match up the 
# date range

In [37]:
mental_health_df.year.min()

'1990'

In [38]:
mental_health_df.year.max()

'2017'

In [39]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47807 entries, 6469 to 54275
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              47807 non-null  int64 
 1   entity             47807 non-null  object
 2   code               46081 non-null  object
 3   year               47807 non-null  object
 4   prevalence_males   6468 non-null   object
 5   prevalance_female  6468 non-null   object
 6   population         46883 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.6+ MB


In [40]:
population_df[population_df.year.str.contains('BCE')]

,index,entity,code,year,prevalence_males,prevalance_female,population
6689,6689,Africa,NaN,10000 BCE,NaN,NaN,241901
6690,6690,Africa,NaN,9000 BCE,NaN,NaN,342782
6691,6691,Africa,NaN,8000 BCE,NaN,NaN,490284
6692,6692,Africa,NaN,7000 BCE,NaN,NaN,707844
6693,6693,Africa,NaN,6000 BCE,NaN,NaN,1031529
...,...,...,...,...,...,...,...
53470,53470,World,OWID_WRL,5000 BCE,NaN,NaN,17920172.000000
53471,53471,World,OWID_WRL,4000 BCE,NaN,NaN,28370428.000000
53472,53472,World,OWID_WRL,3000 BCE,NaN,NaN,44819892.000000
53473,53473,World,OWID_WRL,2000 BCE,NaN,NaN,72108128.000000


In [41]:
population_df['year'] = population_df['year'].str.replace(' BCE', '')

In [42]:
population_df['year'] = population_df.year.astype(int)

In [43]:
# Now the dataset will only include years between 1990 and 2017

population_df = population_df[(population_df['year'] >= 1990) & (population_df['year'] <= 2017)]

In [44]:
population_df

,index,entity,code,year,prevalence_males,prevalance_female,population
6659,6659,Afghanistan,AFG,1990,3.499982,4.647815,12412000.000000
6660,6660,Afghanistan,AFG,1991,3.503947,4.655772,13299000.000000
6661,6661,Afghanistan,AFG,1992,3.508912,4.662066,14486000.000000
6662,6662,Afghanistan,AFG,1993,3.513429,4.669012,15817000.000000
6663,6663,Afghanistan,AFG,1994,3.515578,4.67305,17076000.000000
...,...,...,...,...,...,...,...
54269,54269,Zimbabwe,ZWE,2013,2.769193,3.424106,13350000.000000
54270,54270,Zimbabwe,ZWE,2014,2.778101,3.437674,13587000.000000
54271,54271,Zimbabwe,ZWE,2015,2.789152,3.455323,13815000.000000
54272,54272,Zimbabwe,ZWE,2016,2.799308,3.479071,14030000.000000


In [45]:
s.summarize(population_df)

                    SUMMARY REPORT


Dataframe head: 


,index,entity,code,year,prevalence_males,prevalance_female,population
6659,6659,Afghanistan,AFG,1990,3.499982,4.647815,12412000.000000
6660,6660,Afghanistan,AFG,1991,3.503947,4.655772,13299000.000000
6661,6661,Afghanistan,AFG,1992,3.508912,4.662066,14486000.000000




Dataframe info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7707 entries, 6659 to 54273
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              7707 non-null   int64 
 1   entity             7707 non-null   object
 2   code               6581 non-null   object
 3   year               7707 non-null   int64 
 4   prevalence_males   6468 non-null   object
 5   prevalance_female  6468 non-null   object
 6   population         6783 non-null   object
dtypes: int64(2), object(5)
memory usage: 481.7+ KB


""




Dataframe Description: 


...

,population
6000,51
2000,45
1000,44
10000,42
56000,36
...,...
10315000.000000,1
10534000.000000,1
10645000.000000,1
10746000.000000,1


nulls in dataframe by column: 


,num_rows_missing,percent_rows_missing
prevalence_males,1239,16.076294
prevalance_female,1239,16.076294
code,1126,14.610095
population,924,11.989101
index,0,0.000000
entity,0,0.000000
year,0,0.000000


nulls in dataframe by row: 


,num_cols_missing,percent_cols_missing
41507,3,42.857143
28972,3,42.857143
28963,3,42.857143
28964,3,42.857143
28965,3,42.857143
...,...,...
24590,0,0.000000
24589,0,0.000000
24588,0,0.000000
24587,0,0.000000


In [46]:
population_df[population_df.prevalance_female.isna()].entity.value_counts()

Africa                             29
Asia                               29
Latin America                      29
Europe                             29
Saint Martin (French part)         28
Nauru                              28
New Caledonia                      28
Niue                               28
Palau                              28
Reunion                            28
Saint Barthlemy                    28
Saint Helena                       28
Saint Kitts and Nevis              28
Saint Pierre and Miquelon          28
Monaco                             28
San Marino                         28
Sint Maarten (Dutch part)          28
Tokelau                            28
Turks and Caicos Islands           28
Tuvalu                             28
Vatican                            28
Wallis and Futuna                  28
Western Sahara                     28
Montserrat                         28
Mayotte                            28
Anguilla                           28
Martinique  

In [47]:
# I will drop the null values in this dataset

population_df = population_df.dropna()
population_df.isna().sum()

index                0
entity               0
code                 0
year                 0
prevalence_males     0
prevalance_female    0
population           0
dtype: int64

In [48]:
population_df = population_df.drop(columns='index')

In [49]:
population_df.year.value_counts()

1990    196
1991    196
2016    196
2015    196
2014    196
2013    196
2012    196
2011    196
2010    196
2009    196
2008    196
2007    196
2006    196
2005    196
2004    196
2003    196
2002    196
2001    196
2000    196
1999    196
1998    196
1997    196
1996    196
1995    196
1994    196
1993    196
1992    196
2017    196
Name: year, dtype: int64

In [50]:
mental_health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6468 entries, 0 to 6467
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   entity                 6468 non-null   object 
 1   code                   5488 non-null   object 
 2   year                   6468 non-null   object 
 3   schizophrenia          6468 non-null   object 
 4   bipolar_disorder       6468 non-null   object 
 5   eating_disorders       6468 non-null   object 
 6   anxiety_disorders      6468 non-null   float64
 7   drug_use_disorders     6468 non-null   float64
 8   depression             6468 non-null   float64
 9   alcohol_use_disorders  6468 non-null   float64
dtypes: float64(4), object(6)
memory usage: 505.4+ KB


In [51]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5488 entries, 6659 to 54273
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   entity             5488 non-null   object
 1   code               5488 non-null   object
 2   year               5488 non-null   int64 
 3   prevalence_males   5488 non-null   object
 4   prevalance_female  5488 non-null   object
 5   population         5488 non-null   object
dtypes: int64(1), object(5)
memory usage: 300.1+ KB


In [52]:
# lets change some data types

In [53]:
mental_health_df['year'] = pd.to_datetime(mental_health_df.year)
mental_health_df['year'] = mental_health_df.year.dt.year
mental_health_df['schizophrenia'] = mental_health_df.schizophrenia.astype(float)
mental_health_df['bipolar_disorder'] = mental_health_df.bipolar_disorder.astype(float)
mental_health_df['eating_disorders'] = mental_health_df.eating_disorders.astype(float)

In [54]:
print(mental_health_df.info())
mental_health_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6468 entries, 0 to 6467
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   entity                 6468 non-null   object 
 1   code                   5488 non-null   object 
 2   year                   6468 non-null   int64  
 3   schizophrenia          6468 non-null   float64
 4   bipolar_disorder       6468 non-null   float64
 5   eating_disorders       6468 non-null   float64
 6   anxiety_disorders      6468 non-null   float64
 7   drug_use_disorders     6468 non-null   float64
 8   depression             6468 non-null   float64
 9   alcohol_use_disorders  6468 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 505.4+ KB
None


,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,Afghanistan,AFG,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [55]:
population_df['prevalence_males'] = population_df.prevalence_males.astype(float)
population_df['prevalance_female'] = population_df.prevalance_female.astype(float)
population_df['population'] = population_df.population.astype(float)
population_df['population'] = population_df.population.astype(int)

In [56]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5488 entries, 6659 to 54273
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   entity             5488 non-null   object 
 1   code               5488 non-null   object 
 2   year               5488 non-null   int64  
 3   prevalence_males   5488 non-null   float64
 4   prevalance_female  5488 non-null   float64
 5   population         5488 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 300.1+ KB


In [57]:
mental_health_df.head(3)

,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,Afghanistan,AFG,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644


In [58]:
rates_df.head(3)

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
54276,54276,Entity,Code,Year,"Suicide rate (deaths per 100,000 individuals)",Depressive disorder rates (number suffering pe...,Population,NaN,NaN,NaN,NaN
54277,54277,Afghanistan,AFG,1800,NaN,NaN,3280000,NaN,NaN,NaN,NaN
54278,54278,Afghanistan,AFG,1801,NaN,NaN,3280000,NaN,NaN,NaN,NaN


In [59]:
depressive_rates_df.head(3)

,index,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
102084,102084,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both ...,NaN,NaN,NaN,NaN,NaN,NaN
102085,102085,Afghanistan,AFG,1990,318435.81367,NaN,NaN,NaN,NaN,NaN,NaN
102086,102086,Afghanistan,AFG,1991,329044.773956,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
rates_df = rates_df.drop(columns={'anxiety_disorders', 'drug_use_disorders',
       'depression', 'alcohol_use_disorders'})

In [61]:
depressive_rates_df = depressive_rates_df.drop(columns={'bipolar_disorder',
       'eating_disorders', 'anxiety_disorders', 'drug_use_disorders',
       'depression', 'alcohol_use_disorders'})

In [62]:
rates_df = rates_df.rename(columns={'schizophrenia': 'suicide_rates_per_100k', 
                                    'bipolar_disorder': 'depressive_disorder_rates_per_100k', 
                                    'eating_disorders': 'population'})

In [63]:
rates_df = rates_df.drop(54276)

In [64]:
rates_df

# Im going to change the units from per 100k to a percentage of the population

,index,entity,code,year,suicide_rates_per_100k,depressive_disorder_rates_per_100k,population
54277,54277,Afghanistan,AFG,1800,NaN,NaN,3280000
54278,54278,Afghanistan,AFG,1801,NaN,NaN,3280000
54279,54279,Afghanistan,AFG,1802,NaN,NaN,3280000
54280,54280,Afghanistan,AFG,1803,NaN,NaN,3280000
54281,54281,Afghanistan,AFG,1804,NaN,NaN,3280000
...,...,...,...,...,...,...,...
102079,102079,Zimbabwe,ZWE,2015,27.197061,3068.250731,13815000.0
102080,102080,Zimbabwe,ZWE,2016,26.839591,3081.782858,14030000.0
102081,102081,Zimbabwe,ZWE,2017,26.391769,3094.795065,14237000.0
102082,102082,Zimbabwe,ZWE,2018,NaN,NaN,14439000.0


In [65]:
# ((rates_df.suicide_rates_per_100k / 100_000) * 100)

In [66]:
depressive_rates_df = depressive_rates_df.drop(columns='index')

In [67]:
depressive_rates_df = depressive_rates_df.rename(columns={'schizophrenia': 'prevelance_depressive_disorder'})

In [68]:
depressive_rates_df = depressive_rates_df.drop(102084)

In [69]:
depressive_rates_df

,entity,code,year,prevelance_depressive_disorder
102085,Afghanistan,AFG,1990,318435.81367
102086,Afghanistan,AFG,1991,329044.773956
102087,Afghanistan,AFG,1992,382544.572895
102088,Afghanistan,AFG,1993,440381.507393
102089,Afghanistan,AFG,1994,456916.645489
...,...,...,...,...
108548,Zimbabwe,ZWE,2013,303564.60359
108549,Zimbabwe,ZWE,2014,311665.769283
108550,Zimbabwe,ZWE,2015,320638.507158
108551,Zimbabwe,ZWE,2016,330437.353798


In [70]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47807 entries, 54277 to 102083
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   index                               47807 non-null  int64 
 1   entity                              47807 non-null  object
 2   code                                46081 non-null  object
 3   year                                47807 non-null  object
 4   suicide_rates_per_100k              6468 non-null   object
 5   depressive_disorder_rates_per_100k  6468 non-null   object
 6   population                          46883 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.6+ MB


In [71]:
rates_df = rates_df.drop(columns='index')

In [72]:
rates_df['year'] = rates_df.year.str.replace(' BCE', '')

In [73]:
rates_df['year'] = rates_df.year.astype(int)

In [74]:
rates_df

,entity,code,year,suicide_rates_per_100k,depressive_disorder_rates_per_100k,population
54277,Afghanistan,AFG,1800,NaN,NaN,3280000
54278,Afghanistan,AFG,1801,NaN,NaN,3280000
54279,Afghanistan,AFG,1802,NaN,NaN,3280000
54280,Afghanistan,AFG,1803,NaN,NaN,3280000
54281,Afghanistan,AFG,1804,NaN,NaN,3280000
...,...,...,...,...,...,...
102079,Zimbabwe,ZWE,2015,27.197061,3068.250731,13815000.0
102080,Zimbabwe,ZWE,2016,26.839591,3081.782858,14030000.0
102081,Zimbabwe,ZWE,2017,26.391769,3094.795065,14237000.0
102082,Zimbabwe,ZWE,2018,NaN,NaN,14439000.0


In [75]:
rates_df['suicide_rates_per_100k'] = rates_df.suicide_rates_per_100k.astype(float)
rates_df['depressive_disorder_rates_per_100k'] = rates_df.depressive_disorder_rates_per_100k.astype(float)
rates_df['population'] = rates_df.population.astype(float)

In [76]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47807 entries, 54277 to 102083
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   entity                              47807 non-null  object 
 1   code                                46081 non-null  object 
 2   year                                47807 non-null  int64  
 3   suicide_rates_per_100k              6468 non-null   float64
 4   depressive_disorder_rates_per_100k  6468 non-null   float64
 5   population                          46883 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.2+ MB


In [77]:
depressive_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6468 entries, 102085 to 108552
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   entity                          6468 non-null   object
 1   code                            5488 non-null   object
 2   year                            6468 non-null   object
 3   prevelance_depressive_disorder  6468 non-null   object
dtypes: object(4)
memory usage: 202.3+ KB


In [78]:
depressive_rates_df['year'] = depressive_rates_df.year.str.replace(' BCE', '').astype(int)
depressive_rates_df['year'] = pd.to_datetime(depressive_rates_df.year)

In [79]:
depressive_rates_df['prevelance_depressive_disorder'] = depressive_rates_df.prevelance_depressive_disorder.astype(float)

In [80]:
depressive_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6468 entries, 102085 to 108552
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   entity                          6468 non-null   object        
 1   code                            5488 non-null   object        
 2   year                            6468 non-null   datetime64[ns]
 3   prevelance_depressive_disorder  6468 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 202.3+ KB


In [81]:
print(len(mental_health_df))
mental_health_df.isna().sum()

6468


entity                     0
code                     980
year                       0
schizophrenia              0
bipolar_disorder           0
eating_disorders           0
anxiety_disorders          0
drug_use_disorders         0
depression                 0
alcohol_use_disorders      0
dtype: int64

In [82]:
print(len(population_df))
population_df.isna().sum()

5488


entity               0
code                 0
year                 0
prevalence_males     0
prevalance_female    0
population           0
dtype: int64

In [83]:
print(len(rates_df))
rates_df.isna().sum()

47807


entity                                    0
code                                   1726
year                                      0
suicide_rates_per_100k                41339
depressive_disorder_rates_per_100k    41339
population                              924
dtype: int64

In [84]:
print(len(depressive_rates_df))
depressive_rates_df.isna().sum()

6468


entity                              0
code                              980
year                                0
prevelance_depressive_disorder      0
dtype: int64

In [85]:
# essentially dropping all rows that dont have values

rates_df = rates_df[~rates_df.suicide_rates_per_100k.isna()]
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6468 entries, 54467 to 102081
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   entity                              6468 non-null   object 
 1   code                                5488 non-null   object 
 2   year                                6468 non-null   int64  
 3   suicide_rates_per_100k              6468 non-null   float64
 4   depressive_disorder_rates_per_100k  6468 non-null   float64
 5   population                          5544 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 353.7+ KB


In [86]:
rates_df[rates_df.population.isna()].entity.value_counts()

Andean Latin America                                28
Low SDI                                             28
Western Europe                                      28
Wales                                               28
Tropical Latin America                              28
Sub-Saharan Africa                                  28
Southern Sub-Saharan Africa                         28
Southern Latin America                              28
Southeast Asia, East Asia, and Oceania              28
Southeast Asia                                      28
South Asia                                          28
Scotland                                            28
Northern Ireland                                    28
North Africa and Middle East                        28
Middle SDI                                          28
Low-middle SDI                                      28
Latin America and Caribbean                         28
Australasia                                         28
High-middl

In [87]:
# I am going to drop the null values from this dataset for
# population

rates_df = rates_df[~rates_df.population.isna()]

In [88]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5544 entries, 54467 to 102081
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   entity                              5544 non-null   object 
 1   code                                5488 non-null   object 
 2   year                                5544 non-null   int64  
 3   suicide_rates_per_100k              5544 non-null   float64
 4   depressive_disorder_rates_per_100k  5544 non-null   float64
 5   population                          5544 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 303.2+ KB


In [89]:
# now the only null values should be in the code columns
# lets check

In [90]:
print(rates_df.isna().sum())
print()
print(rates_df.info())
print()
rates_df.head(3)

entity                                 0
code                                  56
year                                   0
suicide_rates_per_100k                 0
depressive_disorder_rates_per_100k     0
population                             0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5544 entries, 54467 to 102081
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   entity                              5544 non-null   object 
 1   code                                5488 non-null   object 
 2   year                                5544 non-null   int64  
 3   suicide_rates_per_100k              5544 non-null   float64
 4   depressive_disorder_rates_per_100k  5544 non-null   float64
 5   population                          5544 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 303.2+ KB
None



,entity,code,year,suicide_rates_per_100k,depressive_disorder_rates_per_100k,population
54467,Afghanistan,AFG,1990,10.318504,4039.755763,12412000.0
54468,Afghanistan,AFG,1991,10.327010,4046.256034,13299000.0
54469,Afghanistan,AFG,1992,10.271411,4053.709902,14486000.0


In [91]:
print(mental_health_df.isna().sum())
print()
print(mental_health_df.info())
print()
mental_health_df.head(3)

entity                     0
code                     980
year                       0
schizophrenia              0
bipolar_disorder           0
eating_disorders           0
anxiety_disorders          0
drug_use_disorders         0
depression                 0
alcohol_use_disorders      0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6468 entries, 0 to 6467
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   entity                 6468 non-null   object 
 1   code                   5488 non-null   object 
 2   year                   6468 non-null   int64  
 3   schizophrenia          6468 non-null   float64
 4   bipolar_disorder       6468 non-null   float64
 5   eating_disorders       6468 non-null   float64
 6   anxiety_disorders      6468 non-null   float64
 7   drug_use_disorders     6468 non-null   float64
 8   depression             6468 non-null   float64
 9   alcohol_use_

,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,Afghanistan,AFG,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644


In [92]:
print(depressive_rates_df.isna().sum())
print()
print(depressive_rates_df.info())
print()
depressive_rates_df.head(3)

entity                              0
code                              980
year                                0
prevelance_depressive_disorder      0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6468 entries, 102085 to 108552
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   entity                          6468 non-null   object        
 1   code                            5488 non-null   object        
 2   year                            6468 non-null   datetime64[ns]
 3   prevelance_depressive_disorder  6468 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 202.3+ KB
None



,entity,code,year,prevelance_depressive_disorder
102085,Afghanistan,AFG,1970-01-01 00:00:00.000001990,318435.813670
102086,Afghanistan,AFG,1970-01-01 00:00:00.000001991,329044.773956
102087,Afghanistan,AFG,1970-01-01 00:00:00.000001992,382544.572895


In [93]:
print(population_df.isna().sum())
print()
print(population_df.info())
print()
population_df.head(3)

entity               0
code                 0
year                 0
prevalence_males     0
prevalance_female    0
population           0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5488 entries, 6659 to 54273
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   entity             5488 non-null   object 
 1   code               5488 non-null   object 
 2   year               5488 non-null   int64  
 3   prevalence_males   5488 non-null   float64
 4   prevalance_female  5488 non-null   float64
 5   population         5488 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 300.1+ KB
None



,entity,code,year,prevalence_males,prevalance_female,population
6659,Afghanistan,AFG,1990,3.499982,4.647815,12412000
6660,Afghanistan,AFG,1991,3.503947,4.655772,13299000
6661,Afghanistan,AFG,1992,3.508912,4.662066,14486000


In [94]:
# okay, still have to fill nulls in code and year needs to be
# just the year and in a datetime type

In [95]:
rates_df.head(1)

,entity,code,year,suicide_rates_per_100k,depressive_disorder_rates_per_100k,population
54467,Afghanistan,AFG,1990,10.318504,4039.755763,12412000.0


In [96]:
rates_df['code'] = rates_df['code'].fillna(rates_df['entity'])
mental_health_df['code'] = mental_health_df['code'].fillna(mental_health_df['entity'])
depressive_rates_df['code'] = depressive_rates_df['code'].fillna(depressive_rates_df['entity'])

In [97]:
print(rates_df.info())
print(mental_health_df.info())
depressive_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5544 entries, 54467 to 102081
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   entity                              5544 non-null   object 
 1   code                                5544 non-null   object 
 2   year                                5544 non-null   int64  
 3   suicide_rates_per_100k              5544 non-null   float64
 4   depressive_disorder_rates_per_100k  5544 non-null   float64
 5   population                          5544 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 303.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6468 entries, 0 to 6467
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   entity                 6468 non-null   object 
 1   code                   6468 non

In [98]:
# Now the big parts

# I will set the index as the year then i will group by the year

In [99]:
mental_health_df.head()

,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,Afghanistan,AFG,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [100]:
# mental_health_df = mental_health_df.set_index('year')

In [101]:
mental_health_df.head()

,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,Afghanistan,AFG,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [102]:
import wrangle as w

In [107]:
mental_health_df, 
population_df, 
rates_df, 
depressive_rates_df = w.separate_data()

In [108]:
mental_health_df

,entity,code,year,schizophrenia,bipolar_disorder,eating_disorders,anxiety_disorders,drug_use_disorders,depression,alcohol_use_disorders
0,Afghanistan,AFG,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260
...,...,...,...,...,...,...,...,...,...,...
6463,Zimbabwe,ZWE,2013,0.155670,0.607993,0.117248,3.090168,0.766280,3.128192,1.515641
6464,Zimbabwe,ZWE,2014,0.155993,0.608610,0.118073,3.093964,0.768914,3.140290,1.515470
6465,Zimbabwe,ZWE,2015,0.156465,0.609363,0.119470,3.098687,0.771802,3.155710,1.514751
6466,Zimbabwe,ZWE,2016,0.157111,0.610234,0.121456,3.104294,0.772275,3.174134,1.513269


In [109]:
population_df

,entity,code,year,prevalence_males,prevalance_female,population
6659,Afghanistan,AFG,1990,3.499982,4.647815,12412000
6660,Afghanistan,AFG,1991,3.503947,4.655772,13299000
6661,Afghanistan,AFG,1992,3.508912,4.662066,14486000
6662,Afghanistan,AFG,1993,3.513429,4.669012,15817000
6663,Afghanistan,AFG,1994,3.515578,4.673050,17076000
...,...,...,...,...,...,...
54269,Zimbabwe,ZWE,2013,2.769193,3.424106,13350000
54270,Zimbabwe,ZWE,2014,2.778101,3.437674,13587000
54271,Zimbabwe,ZWE,2015,2.789152,3.455323,13815000
54272,Zimbabwe,ZWE,2016,2.799308,3.479071,14030000


In [110]:
rates_df

,entity,code,year,suicide_rates_per_100k,depressive_disorder_rates_per_100k,population
54467,Afghanistan,AFG,1990,10.318504,4039.755763,12412000.0
54468,Afghanistan,AFG,1991,10.327010,4046.256034,13299000.0
54469,Afghanistan,AFG,1992,10.271411,4053.709902,14486000.0
54470,Afghanistan,AFG,1993,10.376123,4060.203474,15817000.0
54471,Afghanistan,AFG,1994,10.575915,4062.290365,17076000.0
...,...,...,...,...,...,...
102077,Zimbabwe,ZWE,2013,28.361200,3048.264249,13350000.0
102078,Zimbabwe,ZWE,2014,27.605547,3056.996704,13587000.0
102079,Zimbabwe,ZWE,2015,27.197061,3068.250731,13815000.0
102080,Zimbabwe,ZWE,2016,26.839591,3081.782858,14030000.0


In [111]:
depressive_rates_df

(      index       entity code  year schizophrenia bipolar_disorder  \
 0         0  Afghanistan  AFG  1990       0.16056         0.697779   
 1         1  Afghanistan  AFG  1991      0.160312         0.697961   
 2         2  Afghanistan  AFG  1992      0.160135         0.698107   
 3         3  Afghanistan  AFG  1993      0.160037         0.698257   
 4         4  Afghanistan  AFG  1994      0.160022         0.698469   
 ...     ...          ...  ...   ...           ...              ...   
 6463   6463     Zimbabwe  ZWE  2013       0.15567         0.607993   
 6464   6464     Zimbabwe  ZWE  2014      0.155993          0.60861   
 6465   6465     Zimbabwe  ZWE  2015      0.156465         0.609363   
 6466   6466     Zimbabwe  ZWE  2016      0.157111         0.610234   
 6467   6467     Zimbabwe  ZWE  2017      0.157963         0.611242   
 
      eating_disorders  anxiety_disorders  drug_use_disorders  depression  \
 0            0.101855           4.828830            1.677082    4.07